<a href="https://colab.research.google.com/github/Parkgeunryeong/startproject_colab/blob/main/satrtproweek6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

#  1. 파일 업로드
from google.colab import files
uploaded = files.upload()

# 2. 파일 이름 설정
file_name = "astro.csv"

#  3. pandas로 읽기
df = pd.read_csv(file_name)


#  4. 입력(X)와 출력(y) 분리
X = df[["cloud", "humidity", "precip_prob", "moon_brightness"]]
y = df["score"]

#  5. 스케일링(정규화)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#  6. 학습 데이터 분리 및 모델 훈련
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = RandomForestRegressor()
model.fit(X_train, y_train)

#  7. 모델과 스케일러 저장
joblib.dump(model, "astro_rf_model.pkl")
joblib.dump(scaler, "astro_scaler.pkl")

print(" 모델과 스케일러가 저장되었습니다!")


Saving astro.csv to astro (1).csv
 모델과 스케일러가 저장되었습니다!


In [ ]:
#  설치
!pip install skyfield

import requests
import pandas as pd
import numpy as np
from datetime import datetime
from skyfield.api import load
from skyfield.almanac import moon_phase
import joblib


#  1. 모델과 스케일러 로딩
model = joblib.load("astro_rf_model.pkl")
scaler = joblib.load("astro_scaler.pkl")

#  2. 달 밝기 점수 계산 함수
def get_moon_brightness_score(date_obj):
    eph = load('de421.bsp')
    ts = load.timescale()
    t = ts.utc(date_obj.year, date_obj.month, date_obj.day)
    phase = moon_phase(eph, t).degrees / 360.0
    brightness = round(1 - abs(phase - 0.5) * 2, 2)  # 보름달=1, 신월=0
    return brightness

#  3. OpenWeatherMap API로 5일 예보 가져오기
API_KEY = "7c25d717157701aff47b04889930f2e2"
CITY = "Seoul"
URL = f"https://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={API_KEY}&units=metric&lang=kr"
response = requests.get(URL)

if response.status_code != 200:
    raise Exception("API 요청 실패")

data = response.json()
forecast_list = data["list"]

# 4. 00:00 데이터만 필터링
filtered_data = [entry for entry in forecast_list if "00:00:00" in entry["dt_txt"]]

# 5. 예측 함수
def predict_score(cloud, humidity, pop, moon_brightness):
    input_df = pd.DataFrame([[cloud, humidity, pop, moon_brightness]],
                            columns=["cloud", "humidity", "precip_prob", "moon_brightness"])
    scaled = scaler.transform(input_df)
    score = model.predict(scaled)[0]
    return round(score, 2)

# 6. 각 날짜별 점수 계산 및 출력
print(" 00:00 기준 관측 점수 예측 결과:")
for entry in filtered_data:
    time_str = entry["dt_txt"]
    date_str = time_str.split(" ")[0]
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")

    cloud = entry["clouds"]["all"]
    humidity = entry["main"]["humidity"]
    pop = entry.get("pop", 0.0)
    moon_brightness = get_moon_brightness_score(date_obj)

    score = predict_score(cloud, humidity, pop, moon_brightness)

    print(f"{date_str} → 점수: {score} (달 밝기: {moon_brightness})")



 00:00 기준 관측 점수 예측 결과:
2025-04-09 → 점수: 31.1 (달 밝기: 0.75)
2025-04-10 → 점수: 43.7 (달 밝기: 0.82)
2025-04-11 → 점수: 69.96 (달 밝기: 0.88)
2025-04-12 → 점수: 30.5 (달 밝기: 0.94)
2025-04-13 → 점수: 28.27 (달 밝기: 1.0)
